In [ ]:
!pip install ray
!pip install -U ipywidgets
!pip install pandas
!pip install pyarrow
!pip install ray pyarrow

ERROR: Could not find a version that satisfies the requirement ray-dashboard (from versions: none)
ERROR: No matching distribution found for ray-dashboard


In [ ]:
import ray.data

ds = ray.data.range(5)
ds.show()


2025-05-07 00:18:54,943	ERROR services.py:1362 -- Failed to start the dashboard , return code 3221226505
2025-05-07 00:18:54,943	ERROR services.py:1387 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure' to find where the log file is.
2025-05-07 00:18:54,950	ERROR services.py:1431 -- 
The last 20 lines of C:\Users\Jacky\AppData\Local\Temp\ray\session_2025-05-07_00-18-53_494631_585556\logs\dashboard.log (it contains the error message from the dashboard): 
Traceback (most recent call last):
  File "c:\Users\Jacky\anaconda3\envs\ray\Lib\site-packages\ray\dashboard\dashboard.py", line 247, in <module>
    logging_utils.redirect_stdout_stderr_if_needed(
  File "c:\Users\Jacky\anaconda3\envs\ray\Lib\site-packages\ray\_private\logging_utils.py", line 48, in redirect_stdout_stderr_if_needed
    sys.stderr = open_log(stderr

In [ ]:
import requests
from ray.data.datasource import Datasource, ReadTask
from typing import List
import pandas as pd


class CrisisAPIDatasource(Datasource):
    def prepare_read(self, parallelism: int, **kwargs) -> List[ReadTask]:
        def read():
            # Simulate one partition's data
            response = requests.get("https://api.crisiswatch.org/v1/events")
            data = response.json()["results"]
            return iter([pd.DataFrame(data)])

        return [ReadTask(read, lambda: 1)]

# Load data using the custom API datasource
ds = ray.data.read_datasource(CrisisAPIDatasource())

# Show some of the results
ds.show(limit=5)



2025-05-07 00:09:57,239	ERROR services.py:1362 -- Failed to start the dashboard , return code 3221226505
2025-05-07 00:09:57,240	ERROR services.py:1387 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure' to find where the log file is.
2025-05-07 00:09:57,246	ERROR services.py:1431 -- 
The last 20 lines of C:\Users\Jacky\AppData\Local\Temp\ray\session_2025-05-07_00-09-55_795117_583536\logs\dashboard.log (it contains the error message from the dashboard): 
Traceback (most recent call last):
  File "c:\Users\Jacky\anaconda3\envs\ray\Lib\site-packages\ray\dashboard\dashboard.py", line 247, in <module>
    logging_utils.redirect_stdout_stderr_if_needed(
  File "c:\Users\Jacky\anaconda3\envs\ray\Lib\site-packages\ray\_private\logging_utils.py", line 48, in redirect_stdout_stderr_if_needed
    sys.stderr = open_log(stderr